# Fine Tuning Open AI Models

This notebook is based on the current guidance provided in the [Fine Tuning](https://platform.openai.com/docs/guides/fine-tuning?WT.mc_id=academic-105485-koreyst) documentation from Open AI.

Fine-tuning improves the performance of foundation models for your application by retraining it with additional data and context relevant to that specific use case or scenario. Note that prompt engineering techniques like _few shot learning_ and _retrieval augmented generation_ allow you to enhance the default prompt with relevant data to improve quality. However, these approaches are limited by max token window size of the targeted foundation model. 

With fine-tuning, we are effectively retraining the model itself with the required data (allowing us to use many more examples than can fit in the max token window) - and deploying a _custom_ version of the model that no longer needs to have examples provided at inference time. This not only improves the effectivenenss of our prompt design (we have more flexibility in using the token window for other things) but potentially also improves our costs (by reducing the number of tokens we need to send to the model at inference time).

Fine tuning has 4 steps:
1. Prepare the training data and upload it.
1. Run the training job to get a fine-tuned model.
1. Evaluate the fine-tuned model and iterate for quality.
1. Deploy the fine-tuned model for inference when satisfied.

Note that not all foundation models support fine-tuning - [check OpenAI documentation](https://platform.openai.com/docs/guides/fine-tuning/what-models-can-be-fine-tuned?WT.mc_id=academic-105485-koreyst) for the latest information. You can also fine-tune a previously fine-tuned model. In this tutorial, we'll use `gpt-35-turbo` as our target foundation model for fine-tuning. 

---

### Step 1.1: Prepare Your Dataset

Let's build a chatbot that helps you understand the periodic table of elements by answering questions about an element with a limerick. In _this_ simple tutorial, we'll just create a dataset to train the model with a few sample examples of responses that show the expected format of the data. In a real-world use case, you would need to create a dataset with many more examples. You may also be able to use an open dataset (for your application domain) if one exists, and reformat it for use in fine-tuning.

Since we are focusing on `gpt-35-turbo` and looking for a single-turn response (chat completion) we can create examples using [this suggested format](https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset?WT.mc_id=academic-105485-koreyst) reflecting the OpenAI chat completion requirements. If you expect multi-turn conversational content, you would use the [multi-turn example format](https://platform.openai.com/docs/guides/fine-tuning/multi-turn-chat-examples?WT.mc_id=academic-105485-koreyst) which includes a `weight` parameter to signal which messages should be used (or not) in the fine-tuning process.

We will use the simpler single-turn format for our tutorial here. The data is in the [jsonl format](https://jsonlines.org/?WT.mc_id=academic-105485-koreyst) with 1 record per line, each represented as a JSON-formatted object. The snippet below shows 2 records as a sample - see [training-data.jsonl](training-data.jsonl) for full sample set (10 examples) we'll use for our fine-training tutorial. **Note:** Each record _must_ be defined in a single line (not split across lines as is typical in a formatted JSON file)

```json
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Gallium"}, {"role": "assistant", "content": "Gallium, oh gallium, so light - Melts in your hand, oh what a sight - At 86 degrees - Its liquid with ease - And in semiconductors, it's out of sight"}]}
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Hydrogen"}, {"role": "assistant", "content": "Hydrogen, the first in the line - The lightest of all, so divine - It's in water, you see - And in stars, it's the key - The universe's most common sign"}]}
```

In a real-world use case you will need a much larger examples set for good results - the tradeoff will be between quality of responses and the time/costs for fine-tuning. We are using a small set so we can complete fine-tuning quickly to illustrate the process. See [this OpenAI Cookbook example](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_finetune_chat_models.ipynb?WT.mc_id=academic-105485-koreyst) for a more complex fine-tuning tutorial.

---

### Step 1.2 Upload Your Dataset

Upload the data using the Files API [as described here](https://platform.openai.com/docs/guides/fine-tuning/upload-a-training-file). Note that in order to run this code, you must have done the following steps first:
 - Installed the `openai` Python package (make sure you use a version >=0.28.0 for latest features)
 - Set the `OPENAI_API_KEY` environment variable to your OpenAI API key
To learn more, see the [Setup guide](./../../../00-course-setup/SETUP.md?WT.mc_id=academic-105485-koreyst) provided for the course.

Now, run the code to create a file for upload from your local JSONL file.

In [1]:
from openai import OpenAI
import os
import dotenv

# import dotenv
dotenv.load_dotenv()
api_key = os.getenv('OAI_KEY')
# configure Azure OpenAI service client 

client = OpenAI(api_key=api_key)

ft_file = client.files.create(
  file=open("./training-data.jsonl", "rb"),
  purpose="fine-tune"
)

print(ft_file)
print("Training File ID: " + ft_file.id)

FileObject(id='file-3Gfmyjv8AwbZWHoZABofbF', bytes=4075, created_at=1737521537, filename='training-data.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)
Training File ID: file-3Gfmyjv8AwbZWHoZABofbF


---

### Step 2.1: Create the Fine-tuning job with the SDK

In [8]:
from openai import OpenAI
import dotenv
import os

# import dotenv
dotenv.load_dotenv()

api_key = os.getenv('OAI_KEY')

client = OpenAI(api_key=api_key)

ft_filejob = client.fine_tuning.jobs.create(
  training_file=ft_file.id, 
  model="gpt-3.5-turbo"
)

print(ft_filejob)
print("Fine-tuning Job ID: " + ft_filejob.id)

FineTuningJob(id='ftjob-pDPOLkK8eMMyYoVDg80VqrLj', created_at=1737158003, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(batch_size='auto', learning_rate_multiplier='auto', n_epochs='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-nquYBHFc1JUSUHJAW6Sf7w9Q', result_files=[], seed=1116985646, status='validating_files', trained_tokens=None, training_file='file-PMuPeFBTTU4CeLAucTXxwe', validation_file=None, estimated_finish=None, integrations=[], method=Method(dpo=None, supervised=MethodSupervised(hyperparameters=MethodSupervisedHyperparameters(batch_size='auto', learning_rate_multiplier='auto', n_epochs='auto')), type='supervised'), user_provided_suffix=None)
Fine-tuning Job ID: ftjob-pDPOLkK8eMMyYoVDg80VqrLj


---

### Step 2.2: Check the Status of the job

Here are a few things you can do with the `client.fine_tuning.jobs` API:
- `client.fine_tuning.jobs.list(limit=<n>)` - List the last n fine-tuning jobs
- `client.fine_tuning.jobs.retrieve(<job_id>)` - Get details of a specific fine-tuning job
- `client.fine_tuning.jobs.cancel(<job_id>)` - Cancel a fine-tuning job
- `client.fine_tuning.jobs.list_events(fine_tuning_job_id=<job_id>, limit=<b>)` - List up to n events from the job
- `client.fine_tuning.jobs.create(model="gpt-35-turbo", training_file="your-training-file.jsonl", ...)`

The first step of the process is _validating the training file_ to make sure data is in the right format.

In [2]:
from openai import OpenAI
import os

# import dotenv
dotenv.load_dotenv()

api_key = os.getenv('OAI_KEY')

client = OpenAI(api_key=api_key)

print("1")
# List 10 fine-tuning jobs
client.fine_tuning.jobs.list(limit=10)
print("2")
# Retrieve the state of a fine-tune
client.fine_tuning.jobs.retrieve(ft_filejob.id)
print("3")
# List up to 10 events from a fine-tuning job
client.fine_tuning.jobs.list_events(fine_tuning_job_id=ft_filejob.id, limit=10)

1
2


NameError: name 'ft_filejob' is not defined

In [3]:
# Once the training data is validated
# Track the job status to see if it is running and when it is complete
from openai import OpenAI
import os

# import dotenv
dotenv.load_dotenv()

api_key = os.getenv('OAI_KEY')

client = OpenAI(api_key=api_key)

response = client.fine_tuning.jobs.retrieve(ft_filejob.id)

print("Job ID:", response.id)
print("Status:", response.status)
print("Trained Tokens:", response.trained_tokens)

NameError: name 'ft_filejob' is not defined

---

### Step 2.3: Track events to monitor progress


In [20]:
# You can also track progress in a more granular way by checking for events
# Refresh this code till you get the `The job has successfully completed` message
response = client.fine_tuning.jobs.list_events(ft_filejob.id)

events = response.data
events.reverse()

for event in events:
    print(event.message)

Step 85/100: training loss=0.02
Step 86/100: training loss=0.01
Step 87/100: training loss=0.00
Step 88/100: training loss=0.01
Step 89/100: training loss=0.01
Step 90/100: training loss=0.04
Step 91/100: training loss=0.02
Step 92/100: training loss=0.04
Step 93/100: training loss=0.01
Step 94/100: training loss=0.01
Step 95/100: training loss=0.02
Step 96/100: training loss=0.01
Step 97/100: training loss=0.00
Step 98/100: training loss=0.00
Step 99/100: training loss=0.01
Step 100/100: training loss=0.00
Checkpoint created at step 80
Checkpoint created at step 90
New fine-tuned model created
The job has successfully completed


### Step 2.4: View status in OpenAI Dashboard

You can also view the status by visiting the OpenAI website and exploring the _Fine-tuning_ section of the platform. This will show you the status of the current job, and also let you track the history of prior job execution runs. In this screenshot, you can see that the prior execution failed, and the second run succeeeded. For context, this happened when the first run used a JSON file with incorrectly formatted records  - once fixed, the second run completed successfully and made the model available for use.

![Fine-tuning job status](./img/fine-tuned-model-status.png?WT.mc_id=academic-105485-koreyst)

You can also view the status messages and metrics by scrolling down further in the visual dashboard as shown:

| Messages | Metrics |
|:---|:---|
| ![Messages](./img/fine-tuned-messages-panel.png?WT.mc_id=academic-105485-koreyst) |  ![Metrics](./img/fine-tuned-metrics-panel.png?WT.mc_id=academic-105485-koreyst)|


---

### Step 3.1: Retrieve ID & Test Fine-Tuned Model in Code


In [5]:
# Retrieve the identity of the fine-tuned model once ready
response = client.fine_tuning.jobs.retrieve(ft_filejob.id)
fine_tuned_model_id = response.fine_tuned_model

print("Fine-tuned Model ID:", fine_tuned_model_id)

NameError: name 'ft_filejob' is not defined

In [11]:
# You can then use that model to generate completions from the SDK as shown
# Or you can load that model into the OpenAI Playground (in the UI) to validate it from there.
from openai import OpenAI
import os

# import dotenv
dotenv.load_dotenv()
fine_tuned_model_id="ft:gpt-3.5-turbo-0125:miy-ceramics::AqqZ8I6u"

api_key = os.getenv('OAI_KEY')
print(api_key[:5])
client = OpenAI(api_key=api_key)
print("Fine-tuned Model ID:", fine_tuned_model_id)
completion = client.chat.completions.create(
  model=fine_tuned_model_id,
  messages=[
    {"role": "system", "content": "You are Elle, a factual chatbot that answers questions about elements in the periodic table with a limerick"},
    {"role": "user", "content": "Tell me about Strontium"},
  ]
)
print(completion.choices[0].message)

sk-pr
Fine-tuned Model ID: ft:gpt-3.5-turbo-0125:miy-ceramics::AqqZ8I6u
ChatCompletionMessage(content="Strontium, element number thirty-eight - In Group 2, it just can't wait - Soft metal, silvery white - In fireworks, it's a delight - Burns red - such a dazzling fate", refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None)


In the ceramics industry, a variety of elements are used to achieve different properties and effects. Here are some commonly used elements and their roles:

Silicon (Si) - Glass former, provides structure and strength.
Aluminum (Al) - Stabilizer, increases the durability and temperature resistance of ceramics.
Calcium (Ca) - Alkaline earth metal, acts as a stabilizer and flux, improving the melting properties.
Magnesium (Mg) - Alkaline earth metal, used as a stabilizer and to control the thermal expansion.
Potassium (K) - Alkali metal, serves as a flux to lower the melting point.
Sodium (Na) - Alkali metal, also acts as a flux to lower the melting point.
Boron (B) - Glass former, reduces the melting temperature and improves the flow of glazes.
Titanium (Ti) - Used as an opacifier to make ceramics more opaque and to control crystallization.
Zirconium (Zr) - Stabilizer, improves durability and wear resistance.
Iron (Fe) - Colorant, used to produce various shades of brown, red, and yellow.
Copper (Cu) - Colorant, creates greens and reds in glazes.
Cobalt (Co) - Colorant, produces blue colors in glazes.
Chromium (Cr) - Colorant, used for greens and blacks.
Manganese (Mn) - Colorant, used to produce browns, purples, and blacks.
Nickel (Ni) - Colorant, can produce grays and browns.
Tin (Sn) - Used as an opacifier and can also produce white glazes.
Lead (Pb) - Historically used as a flux in glazes, but its use is now limited due to toxicity concerns.

Each of these elements plays a specific role in the formulation and properties of ceramics and glazes, contributing to the final appearance and performance of the ceramic products.

---

### Step 3.2: Load & Test Fine-Tuned Model in Playground

You can now test the fine-tuned model in two ways. First, you can visit the Playground and use the Models drop-down to select your newly fine-tuned model from the options listed. The other option is to use the "Playground" option shown in the Fine-tuning panel (see screenshot above) which launches the following _comparitive_ view which shows the foundation and fine-tuned model versions side-by-side for quick evaluation.

![Fine-tuning job status](./img/fine-tuned-playground-compare.png?WT.mc_id=academic-105485-koreyst)

Simply fill in the system context used in your training data and provide your test question. You will notice that both sides are updated with the identical context and question. Run the comparison and you will see the difference in outputs between them. _Note how the fine-tuned model renders the response in the format you provided in your examples while the foundation model simply follows the system prompt_.

![Fine-tuning job status](./img/fine-tuned-playground-launch.png?WT.mc_id=academic-105485-koreyst)

You will notice that that the comparison also provides the token counts for each model, and the time taken for the inference. **This specific example is a simplistic one meant to show the process but not actually reflecting a real world dataset or scenario**. You may notice that both samples show the same number of tokens (system context and user prompt are identical) with the fine-tuned model taking more time for inference (custom model).

In real-world scenarios, you will not be using a toy example like this, but fine-tuning against real data (e.g., product catalog for customer service) where the quality of response will be much more apparent. In _that_ context, getting an equivalent response quality with the foundation model will require more custom prompt engineering which will increase token usage and potentially the related processing time for inference. _To try this out, check out the fine-tuning examples in the OpenAI Cookbook to start._

---